<a href="https://colab.research.google.com/github/dennismugane/projo/blob/master/machine_learning_mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%logstop
%logstart -rtq ~/.logs/ml.py append
import seaborn as sns
sns.set()

In [ ]:
from static_grader import grader
from sklearn import set_config

set_config(display="diagram")

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [ ]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
File ‘./ml-data/providers-train.csv’ already there; not retrieving.

File ‘./ml-data/providers-metadata.csv’ already there; not retrieving.



We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation
...,...,...,...,...
74,FINE_TOT,Total Amount of Fines in Dollars,Total Amount of Fines in Dollars,integer
75,PAYDEN_CNT,Number of Payment Denials,Number of Payment Denials,integer
76,TOT_PENLTY_CNT,Total Number of Penalties,Total Number of Penalties,integer
77,LOCATION,Location,Location of facility,only on displayed version of file; renders as ...


In [ ]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')
data.head ()

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
0,015010,COOSA VALLEY NURSING FACILITY,315 WEST HICKORY STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,85,...,Medicare and Medicaid,7,7,0,36,1,0,36,2017-04-06,2016-05-26
1,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,350,Jackson,50,...,Medicare and Medicaid,5,5,0,44,1,0,44,2017-03-16,2016-02-04
2,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,360,Jefferson,92,...,Medicare and Medicaid,6,6,0,40,1,0,40,2016-10-20,2015-12-30
3,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY P O BOX 97,MC CALLA,AL,35111,2054776161,360,Jefferson,103,...,Medicare and Medicaid,2,2,0,16,1,0,16,2017-03-09,2016-02-11
4,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,410,Limestone,149,...,Medicare and Medicaid,2,2,0,20,1,0,20,2017-06-01,2016-05-12


In [ ]:
ss = data.loc[:, ['STATE']]
ss['target'] = fine_totals
v = ss.groupby('STATE')['target'].mean().to_dict()
x = data.loc[:, ['STATE']]
x['y'] = x['STATE'].map(v)
ss[ss['STATE']=='AS']

,STATE,target


## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` method of your estimator, not a DataFrame. This means that your model must work with both data types. You can handle this by adding a test (and optional conversion) in the `predict` method of your custom class, similar to the `ColumnSelectTransformer` given below in Question 2.  

In [ ]:
def not_df(x):
    if isinstance(x, pd.DataFrame):
        return False
    return True

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}
        self.y_mean = 0 

    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        Xt = X.loc[:, ['STATE']]
        Xt['target'] = y
        self.group_averages = Xt.groupby(self.grouper)['target'].mean().to_dict()
        self.y_mean = Xt['target'].mean()
        return self

    def predict(self, X):
        dfp = X
        if not_df(X):
            dfp = pd.DataFrame(X)
        dfp['predicted'] = dfp[self.grouper].map(self.group_averages).fillna(self.y_mean)
        # Return a list of predicted penalties based on group of samples in X
        return dfp['predicted'].tolist()

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [ ]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(steps=[('sme', GroupMeanEstimator(grouper='STATE'))])

Next we should test that our predict method works.

In [ ]:
state_model.predict(data.sample(5))

[9216.964686998395,
 42806.67661691542,
 12365.983606557376,
 8054.977611940299,
 17596.6815920398]

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [ ]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [ ]:
grader.score.ml__state_model(state_model.predict)

Your score: 1.000


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** As mentioned above in Question 1, when the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [ ]:

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns].copy()
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
])
#simple_features.fit_transform(data).info()

**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [ ]:
assert data['RESTOT'].isnull().sum() > 0
#assert not np.isnan(simple_features.fit_transform(data)).any()

In [ ]:
from sklearn.compose import make_column_selector as c_selector
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

def pipe_model(estimator):
    numerical = c_selector(dtype_include=np.number)
    categorical = c_selector(dtype_include=bool)
    num_pipe = Pipeline([('scaler', StandardScaler()), ('simpleimputer', SimpleImputer(strategy='mean'))])
    ct = make_column_transformer((num_pipe, numerical), (OneHotEncoder(handle_unknown = 'ignore'), categorical), remainder = 'passthrough')
    pipe = Pipeline([('simple', simple_features), ('ct', ct), (estimator[0], estimator[1])])
    return pipe

overall_params = {'ct__pipeline__simpleimputer__strategy':['mean', 'median', 'mode', 'constant']}

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

logi=LogisticRegression(random_state=4, n_jobs=-1, solver=  'liblinear', penalty= 'l2', C= 100)
rf=RandomForestClassifier(random_state=4, n_jobs=-1, max_features="auto", warm_start=True, n_estimators= 700, min_samples_leaf= 27, max_depth= 15)
gb=GradientBoostingClassifier(random_state=4, subsample=0.8, max_features="auto", warm_start=True)
dt = DecisionTreeClassifier()

hyper_param = {'logi': {'logi__solver': ['newton-cg', 'lbfgs', 'liblinear'], 'logi__penalty':['l2', 'elasticnet'], 'logi__C':[100, 10, 1.0, 0.1, 0.01]}, 
               'rf': {"rf__max_depth": range(5,30,5), "rf__min_samples_leaf": range(1,30,2), "rf__n_estimators":range(100,2000,200)}, 
               'gb': {"gb__max_depth": range(5,30,5), "gb__min_samples_leaf": range(1,30,2), "gb__n_estimators":range(100,2000,200)},
               'dt': {"dt__max_depth": range(5,30,5), "dt__min_samples_leaf": range(1,30,2), 'dt__min_samples_split': range(10, 110, 10)}}

estimators = {'logi': logi, 'rf': rf, 'gb': gb, 'dt': dt}

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = {}
best_params = {}
for name, estimator in estimators.items():
    gs = RandomizedSearchCV( pipe_model((name, estimator)), param_distributions=dict(overall_params, **hyper_param[name]), 
                            n_iter=10, n_jobs=4, cv=5, scoring='average_precision')
    gs.fit(data, fine_counts > 0)

    scores[name] = gs.best_score_
    best_params[name] = gs.best_params_

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [ ]:
simple_features_model = Pipeline([
    ('simple', simple_features),
    # add your estimator here
    ('pipe', pipe_model(('rf', rf)))
])

In [ ]:
simple_features_model.fit(data, fine_counts > 0)

Pipeline(steps=[('simple',
                 Pipeline(steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL']))])),
                ('pipe',
                 Pipeline(steps=[('simple',
                                  Pipeline(steps=[('cst',
                                                   ColumnSelectTransformer(columns=['BEDCERT',
                                                                                    'RESTOT',
                                                                                    'INHOSP',
                                                                                    'CCRC_FACIL',
                                                                                    'SFF',
                                                                                    'CHOW_LAST_12MOS'...
                                                                                    SimpleImputer())]),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7f85c59d4700>),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   <sklearn.compose._column_transformer.make_column_selector object at 0x7f85c66efe20>)])),
                                 ('rf',
                                  RandomForestClassifier(max_depth=15,
                                                         min_samples_leaf=27,
                                                         n_estimators=700,
                                                         n_jobs=-1,
                                                         random_state=4,
                                                         warm_start=True))]))])

In [ ]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score: 1.078


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [ ]:
from sklearn.pipeline import FeatureUnion

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])), 
    ('one-hot', OneHotEncoder(handle_unknown = 'ignore')),
    ('simpleimputer', SimpleImputer(strategy='mean'))
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])), 
    ('one-hot', OneHotEncoder(handle_unknown = 'ignore')),
    ('simpleimputer', SimpleImputer(strategy='mean'))
])

categorical_features = FeatureUnion([
    ('owner', owner_onehot),
    ('cert', cert_onehot)
])

In [ ]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).astype('float64').dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data)).any()

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore',category=DataConversionWarning)

In [ ]:
from sklearn import model_selection, utils

class_weights = utils.class_weight.compute_class_weight(
    'balanced',np.unique(fine_counts > 0),fine_counts > 0)
classes = np.unique(fine_counts > 0)
class_weights = dict(zip(classes,class_weights))

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[False  True], y=0         True
1        False
2        False
3        False
4        False
         ...  
13887    False
13888    False
13889    False
13890    False
13891     True
Name: FINE_CNT, Length: 13892, dtype: bool as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [ ]:
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    # add your estimator here
    ('clf', DecisionTreeClassifier(random_state=42, max_depth=5, class_weight=class_weights)),
])

In [ ]:
categorical_features_model.fit(data, fine_counts > 0)

Pipeline(steps=[('categorical',
                 FeatureUnion(transformer_list=[('owner',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('one-hot',
                                                                  OneHotEncoder(handle_unknown='ignore')),
                                                                 ('simpleimputer',
                                                                  SimpleImputer())])),
                                                ('cert',
                                                 Pipeline(steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['CERTIFICATION'])),
                                                                 ('one-hot',
                                                                  OneHotEncoder(handle_unknown='ignore')),
                                                                 ('simpleimputer',
                                                                  SimpleImputer())]))])),
                ('clf',
                 DecisionTreeClassifier(class_weight={False: 0.7226383687057845,
                                                      True: 1.6228971962616823},
                                        max_depth=5, random_state=42))])

In [ ]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score: 1.062


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [ ]:
numerical = c_selector(dtype_include=np.number)
categorical = c_selector(dtype_include=bool)
num_pipe = Pipeline([('scaler', StandardScaler()), ('simpleimputer', SimpleImputer(strategy='mean'))])
ct = make_column_transformer((num_pipe, numerical), (OneHotEncoder(handle_unknown = 'ignore'), categorical), remainder = 'passthrough')
simple_pipe = Pipeline([('simple', simple_features), ('ct', ct)])


In [ ]:
business_features = FeatureUnion([
    ('simple', simple_pipe),
    ('categorical', categorical_features)
])

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier
business_model = Pipeline([
    ('features', business_features),
    # add your estimator here
    ('clf', AdaBoostClassifier(n_estimators=50, random_state=42, base_estimator=DecisionTreeClassifier(random_state=42, max_depth=1, class_weight = class_weights)))
])

In [ ]:
business_model.fit(data, fine_counts > 0)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('simple',
                                                 Pipeline(steps=[('simple',
                                                                  Pipeline(steps=[('cst',
                                                                                   ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                    'RESTOT',
                                                                                                                    'INHOSP',
                                                                                                                    'CCRC_FACIL',
                                                                                                                    'SFF',
                                                                                                                    'CHOW_LAST_12MOS',
                                                                                                                    'SPRINKLER_STATUS',
                                                                                                                    'EXP_TOTAL',
                                                                                                                    'ADJ_TOTAL']))])),
                                                                 ('ct',
                                                                  ColumnTransformer(remainder='passthrough',
                                                                                    transformers=[('pipeline',
                                                                                                   Pipeline(st...
                                                                                ('cert',
                                                                                 Pipeline(steps=[('cst',
                                                                                                  ColumnSelectTransformer(columns=['CERTIFICATION'])),
                                                                                                 ('one-hot',
                                                                                                  OneHotEncoder(handle_unknown='ignore')),
                                                                                                 ('simpleimputer',
                                                                                                  SimpleImputer())]))]))])),
                ('clf',
                 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight={False: 0.7226383687057845,
                                                                                        True: 1.6228971962616823},
                                                                          max_depth=1,
                                                                          random_state=42),
                                    random_state=42))])

In [ ]:
grader.score.ml__business_model(positive_probability(business_model))

Your score: 1.025


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [ ]:
data[['CYCLE_1_SURVEY_DATE']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13892 entries, 0 to 13891
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   CYCLE_1_SURVEY_DATE  13892 non-null  object
dtypes: object(1)
memory usage: 108.7+ KB


In [ ]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        Xr = X.copy() # copy() to prevent SettingWithCopyWarning
        Xr['date1'] = pd.to_datetime(Xr[self.t1_col]) 
        Xr['date2'] = pd.to_datetime(Xr[self.t2_col])
        
        return Xr[['date1','date2']].apply(lambda x: (x[0] - x[1]).days, axis=1).to_frame(name='TIMEDELTA')

In [ ]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In [ ]:
data[cycle_1_date]

0        2017-04-06
1        2017-03-16
2        2016-10-20
3        2017-03-09
4        2017-06-01
            ...    
13887    2017-08-10
13888    2017-06-22
13889    2017-09-28
13890    2017-11-09
13891    2017-05-26
Name: CYCLE_1_SURVEY_DATE, Length: 13892, dtype: object

In the cell below we'll collect the cycle 1 survey features.

In [ ]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']

cycle_1_features = Pipeline([
    ('cst', ColumnSelectTransformer(cycle_1_cols)),
    ('scaler', StandardScaler())
])

In [ ]:
from sklearn.ensemble import RandomForestRegressor

survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    # add your estimator here
    ('reg', RandomForestRegressor(random_state=42, max_depth=6, min_samples_split=9, n_estimators=80, max_features=0.1, min_samples_leaf=1)),
])

In [ ]:
survey_model.fit(data, cycle_2_score.astype(int))

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('business',
                                                 FeatureUnion(transformer_list=[('simple',
                                                                                 Pipeline(steps=[('simple',
                                                                                                  Pipeline(steps=[('cst',
                                                                                                                   ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                                    'RESTOT',
                                                                                                                                                    'INHOSP',
                                                                                                                                                    'CCRC_FACIL',
                                                                                                                                                    'SFF',
                                                                                                                                                    'CHOW_LAST_12MOS',
                                                                                                                                                    'SPRINKLER_STATUS',
                                                                                                                                                    'EXP_TOTAL',
                                                                                                                                                    'ADJ_TOTAL']))])),
                                                                                                 ('ct',
                                                                                                  ColumnTransformer(remainder='passthr...
                                                                                                   'CYCLE_1_NFROMDEFS',
                                                                                                   'CYCLE_1_NFROMCOMP',
                                                                                                   'CYCLE_1_DEFS_SCORE',
                                                                                                   'CYCLE_1_NUMREVIS',
                                                                                                   'CYCLE_1_REVISIT_SCORE',
                                                                                                   'CYCLE_1_TOTAL_SCORE'])),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('time',
                                                 TimedeltaTransformer(t1_col='CYCLE_1_SURVEY_DATE',
                                                                      t2_col='CYCLE_2_SURVEY_DATE'))])),
                ('reg',
                 RandomForestRegressor(max_depth=6, max_features=0.1,
                                       min_samples_split=9, n_estimators=80,
                                       random_state=42))])

In [ ]:
grader.score.ml__survey_model(survey_model.predict)

Your score: 1.040


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*